In [1]:
import cv2
import numpy as np
import dlib
import matplotlib.pyplot as plt

In [2]:
def detect_face_landmarks(frame, face_detector, landmark_predictor):
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray_frame)
    
    landmarks_list = []
    for face in faces:
        landmarks = landmark_predictor(gray_frame, face)
        landmarks_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(68)]
        landmarks_list.append(landmarks_points)
    
    return landmarks_list

In [13]:
def extract_eyes_from_landmarks(frame, landmarks):
    
    if len(landmarks) != 68:
        raise ValueError("Facial landmarks should contain 68 points.")

    # Define the indices for the left and right eyes based on facial landmarks.
    left_eye_indices = [i for i in range(36, 42)]
    right_eye_indices = [i for i in range(42, 48)]

    # Extract left and right eye regions from the frame.
    left_eye_coords = np.array([landmarks[i] for i in left_eye_indices], dtype=np.int32)
    right_eye_coords = np.array([landmarks[i] for i in right_eye_indices], dtype=np.int32)

    # Create a mask for each eye region.
    mask_left = np.zeros_like(frame)
    mask_right = np.zeros_like(frame)
    cv2.fillPoly(mask_left, [left_eye_coords], (255, 255, 255))
    cv2.fillPoly(mask_right, [right_eye_coords], (255, 255, 255))

    # Apply the mask to the frame to get the eye regions.
    left_eye_region = cv2.bitwise_and(frame, mask_left)
    right_eye_region = cv2.bitwise_and(frame, mask_right)

    # Convert the eye regions to grayscale for further processing if needed.
    left_eye_gray = cv2.cvtColor(left_eye_region, cv2.COLOR_BGR2GRAY)
    right_eye_gray = cv2.cvtColor(right_eye_region, cv2.COLOR_BGR2GRAY)

    return [left_eye_gray, right_eye_gray]

In [14]:
# Load the dlib face detector and facial landmark predictor models
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('..\\dlib_shape_predictor\\shape_predictor_68_face_landmarks.dat')

# Example usage:
# Assuming 'frame' is your input image
frame=plt.imread("driver.jpg")

# Detect facial landmarks
landmarks = detect_face_landmarks(frame, face_detector, landmark_predictor)

# If a face is detected, extract eye regions from the frame
if len(landmarks) > 0:
    eyes = extract_eyes_from_landmarks(frame, landmarks[0])  # Considering the first detected face
    left_eye, right_eye = eyes[0], eyes[1]


In [16]:
# Open the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Detect facial landmarks
    landmarks = detect_face_landmarks(frame, face_detector, landmark_predictor)

    # If a face is detected, extract eye regions from the frame
    if len(landmarks) > 0:
        eyes = extract_eyes_from_landmarks(frame, landmarks[0])  # Considering the first detected face
        left_eye, right_eye = eyes[0], eyes[1]

        # Display the result
        cv2.imshow('Left Eye', left_eye)
        cv2.imshow('Right Eye', right_eye)
    else:
        cv2.putText(frame, 'No face detected', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow('Frame', frame)

    # Exit the loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()